## Understand what service account is executing this Notebook as additional privileges will need to be given to the service account in order to execute all steps listed here.

In [ ]:
! gcloud auth list

## Enable the following APIs in your project: 
- BigQuery Connection API
- Cloud Functions 

## Also, if you are running everything from a VertexAI Workbench, please make sure the account listed by 'gcloud auth list' has roles:
- BigQuery Admin
- BigQuery Connection Admin
- Cloud Functions Admin 

## Deploy the Cloud Function by running the command below

In [ ]:
! gcloud functions deploy nlp_handler \
--runtime python310 --trigger-http

## Define the variables below ... Change accordinly

In [ ]:
PROJECT_ID="<change-me>"
LOCATION="US"
DATASET="remote_functions"
REMOTE_CONNECTION_NAME="remote_cloudfunc_conn"

## Set gcloud project id

In [ ]:
! gcloud config set project $PROJECT_ID

## Create a BQ remote connection

In [ ]:
! bq mk --connection --display_name='Remote Connection Cloud Function' --connection_type=CLOUD_RESOURCE --project_id=$PROJECT_ID --location=$LOCATION $REMOTE_CONNECTION_NAME

## Create a new BQ dataset for this test 

In [ ]:
! bq --location=$LOCATION mk -d \
    --description "This is my test dataset." \
    $DATASET

## Retrieve the service account identifier associated with the newly created BQ remote connection

In [ ]:
! bq ls --location=$LOCATION --connection $REMOTE_CONNECTION_NAME

## Assign role "Cloud Functions Invoker" to the service account associated with the remote connection above. The command above displays the service account name

## Create a BigQuery Remote UDF, pointing to: 
- The newly created function endpoint
- Use the connection id returned by the command "bq show connection" above

In [ ]:
%%bigquery --project $PROJECT_ID
CREATE OR REPLACE FUNCTION `remote_functions.call_nlp` (x STRING) RETURNS STRING 
REMOTE WITH CONNECTION `<YOUR_REMOTE_CONNECTION_NAME>` 
OPTIONS (endpoint = '<YOUR_FUNCTION_ENDPOINT>', user_defined_context = [("mode","call_nlp")])
;

## Execute a sample SQL statement including the remote UDF

In [ ]:
%%bigquery
select remote_functions.call_nlp("This new feature is fantastic!")

In [ ]:
%%bigquery
SELECT text, remote_functions.call_nlp(text) as score FROM (SELECT text FROM `bigquery-public-data.hacker_news.full` LIMIT 5);
